In [68]:
import re
import pandas as pd
import numpy as np
import datetime

In [69]:
def parser(file):
    #Initialize list
    lines = []
    data1 = []

    #Open File
    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    #Extract Date
    Date = (re.split('[\s]{4,}', lines[1].strip()))
    Date = Date[0]
    Date = Date.replace('RUN DATE:  ', '')

    #Extract indices to cut list
    indices = [i for i, s in enumerate(lines) if '------' in s]
    indicest = [i for i, s in enumerate(lines) if 'NOTE:' in s]

    #Create list with only well licenses
    df1 = lines[(indices[0]+1):(indicest[0]-1)]

    #Create indexes to slice
    df3 = []
    for line in df1:
        df3.extend(re.split('[\s]{2,}', line.strip()))
    
    #Create list without blank items
    df3 = [x for x in df3 if x != '']

    #Create lists for all 7 elements in the table
    df3 = [df3[x:x+7] for x in range(0, len(df3), 7)]

    #Initializes the Dataframe in case the file is empty
    columns = ['LICENCE NUMBER', 'PERMITTEE', 'SUBT CODE', 'PIPELINE FROM LOC', 'O.D. (MAX)', 'APPROVAL DATE', 'OPERATING KM', 'DATE', 'CODE']

    data1 = pd.DataFrame(columns=columns)
    
    #Create a dataframe out of the list
    if df3 != []:
        data1 = pd.DataFrame(df3)
        data1.set_axis(['LICENCE NUMBER', 'PERMITTEE', 'SUBT CODE', 'PIPELINE FROM LOC', 'O.D. (MAX)', 'APPROVAL DATE', 'OPERATING KM'], axis = 1, inplace=True)
        data1['DATE'] = Date

        subt = pd.read_csv('SUBT.csv')
        data1 = pd.merge(data1, subt, how='left', left_on='SUBT CODE', right_on='SUBT CODE',)
        
    return data1

In [70]:
import os

from pathlib import Path

directory_in_str = "/Users/jjayaro/Documents/AER CSVs/ST96"

directory = os.fsencode(directory_in_str) 

columns = ['LICENCE NUMBER', 'PERMITTEE', 'SUBT CODE', 'PIPELINE FROM LOC', 'O.D. (MAX)', 'APPROVAL DATE', 'OPERATING KM', 'DATE']

df = pd.DataFrame(columns=columns)

ext = ['.txt','.TXT']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)):
        #print(filename)
        data = parser(os.path.join(directory_in_str, filename))
        data.to_csv("/Users/jjayaro/Documents/AER CSVs/ST96_CSV/" + filename, index = False)
        #df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

In [13]:
df

,LICENCE NUMBER,PERMITTEE,SUBT CODE,PIPELINE FROM LOC,O.D. (MAX),APPROVAL DATE,OPERATING KM,DATE
0,07670,CANADIAN NATURAL RESOURCES LIMITED,SG,28/067/05/6,323.90,2020-01-02,8.96,02 JAN 2020
1,07671,CANADIAN NATURAL RESOURCES LIMITED,FG,05/067/05/6,114.30,2020-01-02,5.80,02 JAN 2020
2,09577,IMPERIAL OIL RESOURCES LIMITED,SW,05/065/03/4,323.90,2020-01-02,0.41,02 JAN 2020
3,10190,CANADIAN FERTILIZERS LIMITED,NG,14/013/06/4,323.90,2020-01-02,0.00,02 JAN 2020
4,13188,ORLEN UPSTREAM CANADA LTD.,NG,13/039/09/5,219.10,2020-01-02,15.02,02 JAN 2020
...,...,...,...,...,...,...,...,...
6011,60034,NORTH 40 RESOURCES LTD.,NG,05/027/17/4,88.90,2020-10-16,0.00,16 OCT 2020
6012,61899,TALLAHASSEE EXPLORATION INC.,OE,27/122/22/5,88.90,2020-10-16,0.00,16 OCT 2020
6013,61900,TALLAHASSEE EXPLORATION INC.,SW,28/122/22/5,85.60,2020-10-16,0.00,16 OCT 2020
6014,61911,TALLAHASSEE EXPLORATION INC.,CO,18/120/02/6,168.30,2020-10-16,0.00,16 OCT 2020


In [5]:
#Export to CSV
df.to_csv(r'ST96.csv', index = False)